In [1]:
!pip install pandas
!pip install tensorflow
!pip install tensorflow-gpu

In [2]:
import os
import numpy as np
import scipy
import PIL
import tensorflow.keras as keras
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

In [3]:
from collections import defaultdict
import pandas as pd
datasetdir = 'C:\\Users\\mdc20\\Downloads\\img'
os.chdir(datasetdir)
def constructImageClassDataFrame(shape, min_class_occurence = 1, class_list = None):
    # Args: 
    #   Shape: Image shape (2D)
    #   min_class_occurence: number of times class must occur in labeled dir in order to add it to the final sclass list
    
    
    sub_dirs = [d for d in os.listdir(os.getcwd()) if os.path.isdir(os.path.join(datasetdir, d))]
    
    if class_list is None:
        classes = defaultdict(lambda: 0)

        for sub_dir in sub_dirs:
            labels = sub_dir.split('_')
            for label in labels:
                classes[label] += 1
        top_k_classes = [cls for cls in classes if classes[cls] >= min_class_occurence]
        label_classes = top_k_classes    
    else:
        classes = class_list
        label_classes = classes

    
    # classes = list(classes)
    # print("Done calculating list of classes")

    # TODO: remove
    # top_k_classes = ['Graphic', 'Tee', 'Tank', 'Dress', 'Shorts', 'Print', 'Skirt', 'Blouse', 'Top', 'Leggings', 'Sweater', 'Abstract', 'Romper', 'Jumpsuit', 'Floral', 'Cardigan',  'Stripe', 'Boxy','Joggers', 'Striped',  'Tribal', 'Jacket', 'Jeans' ]
    # top_k_classes = ['Top', 'Blouse']

    arr = [[labeled_dir, file] for labeled_dir in sub_dirs for file in os.listdir(labeled_dir) ]
    print("Done parsing through directories")
    df = pd.DataFrame(data=arr, columns=["folder","filename"])



    df['filename'] = df['folder'] + '/' + df['filename']
    df['labels'] = df['folder'].apply(lambda x : [y for y in x.split('_') if y in label_classes] if len([y for y in x.split('_') if y in label_classes]) > 0 else None)

    print(np.shape(df))
    # index_names = df [df['labels'] == []].index
    # Drop Dataframe if none of its labels are present in the selected classes
    df = df[df.labels.notnull()]

    # series_list = list()
    # print("Creating OHE for classes")
    # for idx, cls in enumerate(classes):
    #     if idx %50 == 0: print(idx)
    #     series = df['filename'].apply(lambda x: 1 if cls in x else 0)
    #     series_list.append(series)
    #     # df[str(cls)] = 1 if df['folder'].str.contains(str(cls)) else 0
    
    # df = pd.concat([df, series_list], axis=1)

    #     files = os.listdir(labeled_dir)
    #     labels = labeled_dir.split("_")
    #     for file in files:
    #         continue
            # df.loc[len(df.index)] = [os.path.join(datasetdir, labeled_dir), labels]

    
    # df.head()

    return df, label_classes

# df, top_k_classes = constructImageClassDataFrame(shape = (256,256), min_class_occurence=10,class_list=['Shirt','Shorts'])
# print(df.head(100))
# print(np.shape(df))
# print(top_k_classes)
# print(np.shape(top_k_classes))
    

In [31]:
datasetdir = 'C:\\Users\\mdc20\\Downloads\\img'
os.chdir(datasetdir)

from msilib.schema import Directory
from random import shuffle
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
batch_size = 64



def DataLoad(shape, preprocessing): 
    '''Create the training and validation datasets for 
    a given image shape.
    '''
    imgdatagen = ImageDataGenerator(
        preprocessing_function = preprocessing,
        horizontal_flip = True, 
        validation_split = 0.1,
        rescale = 1.0/255, 
    )

    height, width = shape

    df, classes = constructImageClassDataFrame(shape, class_list=['Shirt','Shorts','Dress','Skirt','Hoodie','Jumpsuit'])

    print(df.sample(n=10))
    # Modify dataframe to only take images with references to classes

    classes = list(classes)
    print("Num classes #")
    print(len(classes)) 
    # for subdir

    train_dataset = imgdatagen.flow_from_dataframe(
        dataframe = df,
        directory = datasetdir,
        x_col="filename",
        y_col="labels",
        batch_size = batch_size,
        seed = 42,
        shuffle = True,
        class_mode="categorical",
        classes = classes,
        subset = 'training'
    )
    val_dataset = imgdatagen.flow_from_dataframe(
        dataframe = df,
        directory = datasetdir,
        x_col="filename",
        y_col="labels",
        batch_size = batch_size,
        seed = 42,
        shuffle = True,
        class_mode="categorical",
        classes = classes,
        subset = 'validation'
    )

    # train_dataset = imgdatagen.flow_from_directory(
    #     os.getcwd(),
    #     target_size = (height, width), 
    #     batch_size = batch_size,
    #     classes = classes,
    #     class_mode = 'categorical',
    #     subset = 'training', 
    # )

    # val_dataset = imgdatagen.flow_from_directory(
    #     os.getcwd(),
    #     target_size = (height, width), 
    #     batch_size = batch_size,
    #     classes = classes,
    #     class_mode = 'categorical',
    #     subset = 'validation'
    # )
    return train_dataset, val_dataset, 

# train_dataset, val_dataset = DataLoad((350,350), preprocessing=preprocess_input)


In [32]:
import scipy
from keras.applications.vgg16 import VGG16, preprocess_input

# Create custom preproocessing routine


train_dataset, val_dataset = DataLoad((256,256), preprocessing=preprocess_input)

print(type(train_dataset))
print(type(val_dataset))
# print(train_dataset.shape())

X_train, y_train = next(train_dataset)

print(type(X_train))
print(y_train[0])
print(np.sum(y_train, axis=1))
print(np.sum(y_train, axis=0))
print(np.shape(X_train))
print(np.shape(y_train))
# print(y_train.shape())    

Done parsing through directories
(289219, 3)
                                         folder  \
145646          High-Waisted_Denim_Pencil_Skirt   
127664                  Floral_Scuba_Knit_Skirt   
165916  Life_in_Progress_Pinstripe_Denim_Shorts   
269133                    Tribal_Print_Jumpsuit   
135061                    Geo_Halter_Maxi_Dress   
179792         Metallic_Chevron-Patterned_Dress   
121865           Floral_Lace_Elasticized_Shorts   
19042                Beaded_Tuxedo_Stripe_Skirt   
196160                 Paisley_Print_Maxi_Skirt   
231558              Sleeveless_Zip-Front_Hoodie   

                                                 filename      labels  
145646   High-Waisted_Denim_Pencil_Skirt/img_00000029.jpg     [Skirt]  
127664           Floral_Scuba_Knit_Skirt/img_00000013.jpg     [Skirt]  
165916  Life_in_Progress_Pinstripe_Denim_Shorts/img_00...    [Shorts]  
269133             Tribal_Print_Jumpsuit/img_00000032.jpg  [Jumpsuit]  
135061             Geo_Halter_Max

In [33]:
y_peek = y_train[:5]

In [34]:
vgg16 = keras.applications.vgg16
conv_model = vgg16.VGG16(weights='imagenet', include_top=False)
conv_model.summary()


Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, None, None, 128)   0     

In [35]:
conv_model = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(256,256,3))

# flatten the output of the convolutional part: 
x = keras.layers.Flatten()(conv_model.output)
# three hidden layers}
x = keras.layers.Dense(100, activation='relu')(x)
x = keras.layers.Dense(100, activation='relu')(x)
x = keras.layers.Dense(100, activation='relu')(x)
# final softmax layer with 40 categories

predictions = keras.layers.Dense(6, activation = 'softmax')(x)
for layer in conv_model.layers:
    layer.trainable = False
# creating the full model:
full_model = keras.models.Model(inputs=conv_model.input, outputs=predictions)
full_model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0   

In [36]:
# from tf.keras.losses import BinaryCrossentropy

cce = tf.keras.losses.CategoricalCrossentropy(reduction=tf.keras.losses.Reduction.SUM, from_logits=False)
# bce = tf.keras.losses.binary_crossentropy

# bce = BinaryCrossentropy()
full_model.compile(loss='categorical_crossentropy',
                  optimizer=keras.optimizers.Adamax(learning_rate=0.001),
                  metrics=['acc', tf.keras.metrics.CategoricalCrossentropy()])

In [37]:
# Configure run environment
from tensorflow.python.client import device_lib

device_lib.list_local_devices()
# tf.config.list_physical_devices('CPU')

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 14644894508509338167
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 5762973696
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 5296306321352949287
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3060 Ti, pci bus id: 0000:01:00.0, compute capability: 8.6"
 xla_global_id: 416903419]

In [38]:
from plot_keras_history import show_history, plot_history
history = full_model.fit(
    train_dataset, 
    validation_data = val_dataset,
    workers=1,
    epochs=10,
)

full_model.save('type_kernel.keras_model')

Epoch 1/10
1647/1647 [==============================] - 673s 405ms/step - loss: 0.8886 - acc: 0.7543 - categorical_crossentropy: 0.8886 - val_loss: 1.5232 - val_acc: 0.7742 - val_categorical_crossentropy: 1.5232
Epoch 2/10
1647/1647 [==============================] - 683s 414ms/step - loss: 5.9185 - acc: 0.6518 - categorical_crossentropy: 5.9185 - val_loss: 16.8796 - val_acc: 0.5027 - val_categorical_crossentropy: 16.8796
Epoch 3/10
1303/1647 [======================>.......] - ETA: 2:06 - loss: 21.5287 - acc: 0.5891 - categorical_crossentropy: 21.5287

In [13]:
history3 = full_model.fit(
    train_dataset, 
    validation_data = val_dataset,
    workers=1,
    epochs=20,
)

Epoch 1/20
10392/10392 [==============================] - 186s 18ms/step - loss: 0.5118 - acc: 0.7635 - categorical_crossentropy: 0.5118 - val_loss: 0.8866 - val_acc: 0.4163 - val_categorical_crossentropy: 0.8866
Epoch 2/20
10392/10392 [==============================] - 184s 18ms/step - loss: 0.4846 - acc: 0.7754 - categorical_crossentropy: 0.4846 - val_loss: 0.7567 - val_acc: 0.5193 - val_categorical_crossentropy: 0.7567
Epoch 3/20
10392/10392 [==============================] - 183s 18ms/step - loss: 0.4709 - acc: 0.7823 - categorical_crossentropy: 0.4709 - val_loss: 1.0742 - val_acc: 0.4362 - val_categorical_crossentropy: 1.0742
Epoch 4/20
10392/10392 [==============================] - 183s 18ms/step - loss: 0.4617 - acc: 0.7865 - categorical_crossentropy: 0.4617 - val_loss: 0.9494 - val_acc: 0.4645 - val_categorical_crossentropy: 0.9494
Epoch 5/20
10392/10392 [==============================] - 182s 18ms/step - loss: 0.4522 - acc: 0.7917 - categorical_crossentropy: 0.4522 - val_loss:

In [ ]:
# full_model.save('sparse_categorical_cross_entropy.h5')
full_model.save('39_acc.h5')

In [ ]:
from keras.models import load_model
model = load_model('cat_cross_entropy_16_classes.h5')

In [ ]:
from sklearn.metrics import confusion_matrix

x_val, y_val = next(val_dataset)
y_prediction = model.predict(x_val)
print(y_prediction[2])
print(y_val[2])
print(np.shape(y_prediction))
# preds = np.argmax(y_prediction, axis=1)
# print(preds)

k = 4
axis = 1 # Search for top k values in row
top_k_indx = np.argpartition(y_prediction, -4, axis=axis)[-4:].T
print(top_k_indx)
print(np.shape(top_k_indx))
# result = confusion_matrix(y_val, y_prediction, normalize='y_pred=')
# print(y_prediction)